In [25]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import sys 
import math

In [26]:
from __future__ import print_function
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import plot_model
from keras.layers.recurrent import LSTM
from keras.callbacks import EarlyStopping
from keras.initializers import glorot_uniform
from keras.initializers import orthogonal
from keras.initializers import TruncatedNormal

In [27]:
df = pd.read_csv('x-data.csv')
#print(df.iloc[:,:])
# nullを含んだ行を削除
# https://note.nkmk.me/python-pandas-nan-dropna-fillna/
df2 = df.iloc[:,:].dropna(how='any')
buff = df2.iloc[:,-1] != 0.0
df3 = df2[buff]
print(df3.dtypes) 
print(df3)

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
dtype: object
             Date          Open          High           Low         Close  \
9764   2002-06-10  11470.919922  11522.040039  11370.209961  11370.209961   
9765   2002-06-11  11390.410156  11514.530273  11390.410156  11449.440430   
9766   2002-06-12  11392.320313  11405.290039  11261.929688  11327.059570   
9767   2002-06-13  11366.059570  11396.280273  11132.589844  11144.839844   
9768   2002-06-14  11121.889648  11127.160156  10911.070313  10920.629883   
...           ...           ...           ...           ...           ...   
14225  2020-04-16  19311.300781  19362.169922  19154.410156  19290.199219   
14226  2020-04-17  19575.849609  19922.070313  19554.699219  19897.259766   
14227  2020-04-20  19689.849609  19784.380859  19611.789063  19669.119141   
14228  2020-04-21  19479.830078  19529.060547  19193.220703  19280.77

In [28]:
df4 = np.log(df3.iloc[:,4])
df5 = (df4.diff())
print (df5*100)
df3.Close = df5*100

9764          NaN
9765     0.694408
9766    -1.074634
9767    -1.621792
9768    -2.032295
           ...   
14225   -1.338273
14226    3.098487
14227   -1.153217
14228   -1.994114
14229   -0.743547
Name: Close, Length: 4366, dtype: float64


C:\Users\junde\miniconda3\envs\dl4us\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [29]:
train_df = df3.iloc[:,[0,4]]
label_df = buf.drop(9764)
print(train_df)
date_df = train_df.Date

             Date     Close
9764   2002-06-10       NaN
9765   2002-06-11  0.694408
9766   2002-06-12 -1.074634
9767   2002-06-13 -1.621792
9768   2002-06-14 -2.032295
...           ...       ...
14225  2020-04-16 -1.338273
14226  2020-04-17  3.098487
14227  2020-04-20 -1.153217
14228  2020-04-21 -1.994114
14229  2020-04-22 -0.743547

[4366 rows x 2 columns]


In [30]:
more_1_plus = train_df.Close > math.log(1.01)*100
less_1_plus = (train_df.Close < math.log(1.01) * 100)&(train_df.Close > math.log(1.00) * 100)
less_1_minus = (train_df.Close < math.log(1.00)*100) & (train_df.Close > math.log(0.99)* 100 )
more_1_minus = train_df.Close < math.log(0.99) * 100

'''
more_1_plus = more_1_plus.index.rename('x>1')
less_1_plus = less_1_plus.index.rename('0<=x<1')
less_1_minus = less_1_minus.index.rename('-1<x<0')
more_1_minus = more_1_minus.index.rename('x<-1')
'''

more_1_plus = more_1_plus*1
less_1_plus = less_1_plus*1
less_1_minus = less_1_minus*1
more_1_minus = more_1_minus*1


##print(more_1_plus*1)
##print(less_1_plus)
##print(less_1_minus)
##print(more_1_minus)

In [31]:
buf = pd.concat([date_df,more_1_plus], axis=1)
buf = pd.concat([buf,less_1_plus], axis=1)
buf = pd.concat([buf, less_1_minus], axis=1)
buf = pd.concat([buf, more_1_minus], axis=1)

In [32]:
buf.columns = ['Date','more_1_plus','less_1_plus','less_1_minus','more_1_minus']
print(buf.columns)
print(buf)


Index(['Date', 'more_1_plus', 'less_1_plus', 'less_1_minus', 'more_1_minus'], dtype='object')
             Date  more_1_plus  less_1_plus  less_1_minus  more_1_minus
9764   2002-06-10            0            0             0             0
9765   2002-06-11            0            1             0             0
9766   2002-06-12            0            0             0             1
9767   2002-06-13            0            0             0             1
9768   2002-06-14            0            0             0             1
...           ...          ...          ...           ...           ...
14225  2020-04-16            0            0             0             1
14226  2020-04-17            1            0             0             0
14227  2020-04-20            0            0             0             1
14228  2020-04-21            0            0             0             1
14229  2020-04-22            0            0             1             0

[4366 rows x 5 columns]


In [33]:
train_df = train_df.drop(9764)
label_df = buf.drop(9764)

In [34]:
print(train_df)
print(label_df)

             Date     Close
9765   2002-06-11  0.694408
9766   2002-06-12 -1.074634
9767   2002-06-13 -1.621792
9768   2002-06-14 -2.032295
9769   2002-06-17 -2.376972
...           ...       ...
14225  2020-04-16 -1.338273
14226  2020-04-17  3.098487
14227  2020-04-20 -1.153217
14228  2020-04-21 -1.994114
14229  2020-04-22 -0.743547

[4365 rows x 2 columns]
             Date  more_1_plus  less_1_plus  less_1_minus  more_1_minus
9765   2002-06-11            0            1             0             0
9766   2002-06-12            0            0             0             1
9767   2002-06-13            0            0             0             1
9768   2002-06-14            0            0             0             1
9769   2002-06-17            0            0             0             1
...           ...          ...          ...           ...           ...
14225  2020-04-16            0            0             0             1
14226  2020-04-17            1            0             0      

In [36]:
train_df.to_csv("train.csv")
label_df.to_csv("label.csv")

In [47]:
df1 = csv.reader(open('train.csv', 'r'))
data1 = [ v for v in df1]
mat = np.array(data1)
mat2 = mat[1:]                        # 見出し行を外す
x_data = mat2[:, 2:].astype(np.float)  # 2列目以降を抜き出してfloat変換
print('x_data.shape=', x_data.shape)

x_data.shape= (4365, 1)


In [48]:
df2 = csv.reader(open('label.csv', 'r'))
data2 = [ v for v in df2]
mat3 = np.array(data2)
mat4 = mat3[1:]                       # 見出し行を外す
t_data = mat4[:, 2:].astype(np.float)  # 2列目以降を抜き出してfloat変換
print('t_data.shape=', t_data.shape)

t_data.shape= (4365, 4)


In [49]:
maxlen = 80              # 入力系列数
n_in = x_data.shape[1]   # 学習データ（＝入力）の列数
n_out = t_data.shape[1]  # ラベルデータ（=出力）の列数
len_seq = x_data.shape[0] - maxlen + 1
data = []
target = []
for i in range(0, len_seq):
  data.append(x_data[i:i+maxlen, :])
  target.append(t_data[i+maxlen-1, :])

x = np.array(data).reshape(len(data), maxlen, n_in)
t = np.array(target).reshape(len(data), n_out)

print(x.shape, t.shape)

# ここからソースコードの後半
n_train = int(len(data)*0.9)              # 訓練データ長
x_train,x_test = np.vsplit(x, [n_train])  # 学習データを訓練用とテスト用に分割
t_train,t_test = np.vsplit(t, [n_train])  # ラベルデータを訓練用とテスト用に分割

print(x_train.shape, x_test.shape, t_train.shape, t_test.shape)

(4286, 80, 1) (4286, 4)
(3857, 80, 1) (429, 80, 1) (3857, 4) (429, 4)


In [65]:
print(len(data))
print(t_data[0,:])

4286
[0. 1. 0. 0.]


In [50]:
class Prediction :
  def __init__(self, maxlen, n_hidden, n_in, n_out):
    self.maxlen = maxlen
    self.n_hidden = n_hidden
    self.n_in = n_in
    self.n_out = n_out

  def create_model(self):
    model = Sequential()
    model.add(LSTM(self.n_hidden, batch_input_shape = (None, self.maxlen, self.n_in),
             kernel_initializer = glorot_uniform(seed=20170719), 
             recurrent_initializer = orthogonal(gain=1.0, seed=20170719), 
             dropout = 0.5, 
             recurrent_dropout = 0.5))
    model.add(Dropout(0.5))
    model.add(Dense(self.n_out, 
            kernel_initializer = glorot_uniform(seed=20170719)))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer = "RMSprop", metrics = ['categorical_accuracy'])
    return model

  # 学習
  def train(self, x_train, t_train, batch_size, epochs) :
    early_stopping = EarlyStopping(patience=0, verbose=1)
    model = self.create_model()
    model.fit(x_train, t_train, batch_size = batch_size, epochs = epochs, verbose = 1,
          shuffle = True, callbacks = [early_stopping], validation_split = 0.1)
    return model

In [52]:
n_hidden = 80     # 出力次元
epochs = 100      # エポック数
batch_size = 10   # ミニバッチサイズ

# モデル定義
prediction = Prediction(maxlen, n_hidden, n_in, n_out)
# 学習
model = prediction.train(x_train, t_train, batch_size, epochs)
# テスト
score = model.evaluate(x_test, t_test, batch_size = batch_size, verbose = 1)
print("score:", score)

# 正答率、準正答率（騰落）集計
preds = model.predict(x_test)
correct = 0
semi_correct = 0
for i in range(len(preds)):
  pred = np.argmax(preds[i,:])
  tar = np.argmax(t_test[i,:])
  if pred == tar :
    correct += 1
  else :
    if pred+tar == 1 or pred+tar == 5 :
      semi_correct += 1

print("正答率:", 1.0 * correct / len(preds))
print("準正答率（騰落）:", 1.0 * (correct+semi_correct) / len(preds))

Train on 3471 samples, validate on 386 samples
Epoch 1/100
3471/3471 [==============================] - 8s 2ms/step - loss: 1.2269 - categorical_accuracy: 0.4180 - val_loss: 1.0418 - val_categorical_accuracy: 0.7720
Epoch 2/100
3471/3471 [==============================] - 8s 2ms/step - loss: 1.0594 - categorical_accuracy: 0.5076 - val_loss: 0.8801 - val_categorical_accuracy: 0.8627
Epoch 3/100
3471/3471 [==============================] - 8s 2ms/step - loss: 0.9582 - categorical_accuracy: 0.5494 - val_loss: 0.7120 - val_categorical_accuracy: 0.8212
Epoch 4/100
3471/3471 [==============================] - 9s 2ms/step - loss: 0.9168 - categorical_accuracy: 0.5667 - val_loss: 0.6784 - val_categorical_accuracy: 0.8264
Epoch 5/100
3471/3471 [==============================] - 8s 2ms/step - loss: 0.8803 - categorical_accuracy: 0.5733 - val_loss: 0.5781 - val_categorical_accuracy: 0.8135
Epoch 6/100
3471/3471 [==============================] - 9s 2ms/step - loss: 0.8486 - categorical_accuracy: 